In [ ]:
!pip install pandas scikit-learn matplotlib seaborn flask openpyxl joblib


# load dataset

In [ ]:

from google.colab import files
import pandas as pd

df = pd.read_excel('/content/sample_data/tourist_places_bd_real_desc.xlsx')
print(df.shape)
df.head()


(430, 9)


,place_name,category,budget_level,season,trip_type,duration_days,popularity,location,description
0,Saint Martin's Island,island,high,winter,family,6,7.0,"Bay of Bengal, Teknaf",Saint Martin's Island is Bangladesh's only cor...
1,Bashundhara City,cultural,high,summer,family,5,7.1,"Panthapath, Dhaka",Bashundhara City is one of the largest shoppin...
2,Baitul Mukarram Mosque,religious,high,winter,solo,6,8.4,Dhaka,"Baitul Mukarram Mosque, located in the heart o..."
3,Bhawal National Park,forest,high,winter,family,1,9.0,Gazipur,Bhawal National Park is a serene forest reserv...
4,Paharpur (Somapura Mahavihara),historical,high,summer,family,6,9.1,Naogaon,"Paharpur, home to the ancient Somapura Mahavih..."


# Data preprocessing

In [ ]:

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

df.dropna(inplace=True)

# Encode categorical columns
categorical_cols = ['category', 'budget_level', 'season', 'trip_type']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Scale duration
scaler = StandardScaler()
df['duration_days_scaled'] = scaler.fit_transform(df[['duration_days']])

# Clustering to Group Similar Tourist Places

In [ ]:

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

features = df[['category', 'budget_level', 'season', 'trip_type', 'duration_days_scaled']]
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features)

kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(features_pca)

silhouette_avg = silhouette_score(features_pca, df['cluster'])
print(f"Silhouette Score: {silhouette_avg}")

Silhouette Score: 0.4053889513678745


# Test split

In [ ]:

X = df[['category', 'budget_level', 'season', 'trip_type', 'duration_days_scaled']]
y = df['cluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Model Training and Evaluation

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB()
}

results = {}
cv_results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    results[name] = {'Accuracy': acc, 'F1 Score': f1}

    cv_f1 = cross_val_score(model, X, y, cv=5, scoring='f1_weighted')
    cv_acc = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    cv_results[name] = {
        'Mean F1 Score': np.mean(cv_f1),
        'Std F1 Score': np.std(cv_f1),
        'Mean Accuracy': np.mean(cv_acc),
        'Std Accuracy': np.std(cv_acc)
    }

print(pd.DataFrame(results).T)
print(pd.DataFrame(cv_results).T.sort_values(by="Mean F1 Score", ascending=False))


                     Accuracy  F1 Score
Logistic Regression  0.976744  0.976496
Decision Tree        1.000000  1.000000
Random Forest        0.988372  0.988323
SVM                  0.930233  0.931470
KNN                  0.941860  0.942764
Naive Bayes          0.790698  0.795704
                     Mean F1 Score  Std F1 Score  Mean Accuracy  Std Accuracy
Logistic Regression       0.990638      0.013688       0.990698      0.013560
Decision Tree             0.974298      0.017237       0.979070      0.008702
Random Forest             0.974123      0.029384       0.974419      0.023716
SVM                       0.967552      0.008756       0.967442      0.008702
KNN                       0.955881      0.031389       0.955814      0.031546
Naive Bayes               0.748056      0.030409       0.753488      0.029782


# Save Best Model and Preprocessors

In [ ]:
import joblib

# Save the best model
joblib.dump(models['Logistic Regression'], 'final_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')
joblib.dump(kmeans, 'kmeans.pkl')  # Optional if you want clustering again
joblib.dump(df, 'full_df_with_descriptions.pkl')  # For Flask use

from google.colab import files
files.download('final_model.pkl')
files.download('scaler.pkl')
files.download('label_encoders.pkl')
files.download('full_df_with_descriptions.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>